Using methodology from https://automaticaddison.com/the-ultimate-guide-to-real-time-lane-detection-using-opencv/ adapting to our usecase

In [77]:
import cv2 as cv 
import pandas as pd
import numpy as np

In [61]:
img = cv.imread(r'testMaps/forest_from_above.jpg')

In [62]:
# cv.imshow('Original', img)
# cv.waitKey(0)

In [63]:
imgHLS = cv.cvtColor(img, cv.COLOR_BGR2HLS)  # Convert image from BGR (full color) to HLS (hue, saturation, lightness)
# cv.imshow('hls', imgHLS)
# cv.imwrite(r"testMaps/tmp/HLS.jpg", imgHLS)

sobelBinary thresholds the Lightness channel to try to find sharp discontinuities in pixel intensities, GuassianBlur is used to reduce noise.

In [107]:
## Try to find road edges

sobelBinary = cv.threshold(imgHLS[:, :, 1], 155, 255, cv.THRESH_BINARY)[1]  # binary threshold on lightness channel
cv.imwrite(r"testMaps/tmp/sobelBinary.jpg", sobelBinary)
sobelBinary = cv.GaussianBlur(sobelBinary, (3, 3), 0)  # to reduce noise
cv.imwrite(r"testMaps/tmp/sobelBinaryGuassian.jpg", sobelBinary)

sobelx = np.absolute(cv.Sobel(sobelBinary, ddepth=cv.CV_64F, dx=1, dy=0, ksize=3))
sobely = np.absolute(cv.Sobel(sobelBinary, ddepth=cv.CV_64F, dx=0, dy=1, ksize=3))
mags = np.sqrt(sobelx ** 2 + sobely ** 2)
binary = np.ones_like(mags)  # array of 1's with same shape/type as mags
binary[(mags >= 155) & (mags <= 255)] = 255
 
sobelBinary = binary
cv.imwrite(r"testMaps/tmp/sobelBinaryMags.jpg", sobelBinary)

True

In [135]:
## Isolate lanes 

sBinary = cv.threshold(imgHLS[:, :, 2], 155, 255, cv.THRESH_BINARY)[1]  # binary threshold on saturation channel
cv.imwrite(r"testMaps/tmp/sBinary.jpg", sBinary)

#green mask below is useful for forests, should make more generalizable for future work though
greenThresh = cv.threshold(img[:, :, 1], 145, 255, cv.THRESH_BINARY_INV)[1]  # if original img has green pixel value > 155, make it black, else make it white
cv.imwrite(r"testMaps/tmp/greenThresh.jpg", greenThresh)

bitBinary = cv.bitwise_and(sBinary, greenThresh)
cv.imwrite(r"testMaps/tmp/bitBinary.jpg", bitBinary)

# showRoads = cv.bitwise_or(bitBinary, sobelBinary.astype(np.uint8))  #using bitBinary
# cv.imwrite(r"testMaps/tmp/showRoads.jpg", showRoads)

showRoads = cv.bitwise_or(greenThresh, sobelBinary.astype(np.uint8))  #using greenThresh
cv.imwrite(r"testMaps/tmp/showRoads.jpg", showRoads)

True

For us here, greenThresh is superior as it just gets rid of all the trees.  Not generalizable though.